# **Bitcoin price prediction - Gradient Boosting Tree Regressor (Walk Forward Split)**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author: Corsi Danilo (1742375) - corsi.1742375@studenti.uniroma1.it


---


Description: perform model's train / validation with hyperparameter tuning and cross validation based on different methods of splitting the dataset.

# Global constants, dependencies, libraries and tools

In [1]:
# Main constants
LOCAL_RUNNING = True
SLOW_OPERATIONS = False # Decide whether or not to use operations that might slow down notebook execution
ROOT_DIR = "D:/Documents/Repository/BDC/project" if LOCAL_RUNNING else "/content/drive"

In [2]:
if not LOCAL_RUNNING:
    # Point Colaboratory to Google Drive
    from google.colab import drive

    # Define GDrive paths
    drive.mount(ROOT_DIR, force_remount=True)

    # Install Spark and related dependencies
    !pip install pyspark
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

## Import my utilities

In [3]:
# Set main dir
MAIN_DIR = ROOT_DIR + "" if LOCAL_RUNNING else ROOT_DIR + "/MyDrive/BDC/project"

# Utilities dir
UTILITIES_DIR = MAIN_DIR + "/utilities"

# Import my utilities
import sys
sys.path.append(UTILITIES_DIR)

from imports import *
import train_validation_utilities
from config import *

importlib.reload(train_validation_utilities)

<module 'train_validation_utilities' from 'D:\\Documents/Repository/BDC/project/utilities\\train_validation_utilities.py'>

## Core variables

In [4]:
# BS = Block Split
# WFS = Walk Forward Split
# SS = Single Split
SPLITTING_METHOD = WFS

# LR = LinearRegression 
# GLR = GeneralizedLinearRegression 
# RF = RandomForestRegressor 
# GBTR = GradientBoostingTreeRegressor
MODEL_NAME = GBTR

In [5]:
###################
# --- DATASET --- #
###################

# Datasets dirs
DATASET_OUTPUT_DIR = MAIN_DIR + "/datasets/output"

# Datasets paths
DATASET_TRAIN_VALID  = DATASET_OUTPUT_DIR + "/" + DATASET_TRAIN_VALID_NAME + ".parquet"

####################
# --- FEATURES --- #
####################

# Features dir
FEATURES_DIR = MAIN_DIR + "/features"

# Features paths
FEATURES_CORRELATION = FEATURES_DIR + "/" + FEATURES_CORRELATION_LABEL + ".json"
BASE_FEATURES = FEATURES_DIR + "/" + BASE_FEATURES_LABEL + ".json"
BASE_AND_MOST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_MOST_CORR_FEATURES_LABEL + ".json"
BASE_AND_LEAST_CORR_FEATURES = FEATURES_DIR + "/" + BASE_AND_LEAST_CORR_FEATURES_LABEL + ".json"

##################
# --- MODELS --- #
##################

# Model dir
MODELS_DIR = MAIN_DIR + "/models"

# Model path
MODEL = MODELS_DIR + "/" + MODEL_NAME

###################
# --- RESULTS --- #
###################

# Results dir
RESULTS_DIR = MAIN_DIR + "/results/" + SPLITTING_METHOD

# Results path
ALL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_all.csv"
REL_MODEL_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_rel.csv"

MODEL_ACCURACY_RESULTS  = RESULTS_DIR + "/" + MODEL_NAME + "_accuracy.csv"

In [6]:
# Importing useful libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pio.renderers.default = 'vscode+colab' # To correctly render plotly plots

# Create the pyspark session

In [7]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '109G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPricePrediction").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

# Loading dataset

In [8]:
# Load train / validation set into pyspark dataset objects
df = spark.read.load(DATASET_TRAIN_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [9]:
train_validation_utilities.dataset_info(df)

+-------------------+---+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+------------------------+-------------------+------------------+--------------------+--------------------+------------------+--------------+--------------------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|          timestamp| id|     market-price|    opening-price|    highest-price|     lowest-price|    closing-price|  trade-volume-btc|   total-bitcoins|          market-cap|    trade-volume-usd|       blocks-size|    avg-block-size|n-transactions-total|n-transactions-per-block|          hash-rate|        difficulty|      miners-revenue|transaction-fees-usd|n-unique-addresses|n-transactions|estimated-transaction-volume-usd|       sma-5-days|   

# Loading features

In [10]:
# Loading base features
with open(BASE_FEATURES, "r") as f:
    BASE_FEATURES = json.load(f)
print(BASE_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd']


In [11]:
# Loading currency and additional most correlated features
with open(BASE_AND_MOST_CORR_FEATURES, "r") as f:
    BASE_AND_MOST_CORR_FEATURES = json.load(f)
print(BASE_AND_MOST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'miners-revenue', 'sma-5-days', 'sma-7-days', 'sma-10-days', 'estimated-transaction-volume-usd', 'sma-20-days']


In [12]:
# Loading currency and additional least correlated features
with open(BASE_AND_LEAST_CORR_FEATURES, "r") as f:
    BASE_AND_LEAST_CORR_FEATURES = json.load(f)
print(BASE_AND_LEAST_CORR_FEATURES)

['opening-price', 'highest-price', 'lowest-price', 'closing-price', 'trade-volume-btc', 'market-price', 'market-cap', 'total-bitcoins', 'trade-volume-usd', 'sma-100-days', 'transaction-fees-usd', 'n-unique-addresses', 'sma-50-days', 'n-transactions-total', 'blocks-size', 'hash-rate', 'difficulty', 'avg-block-size', 'n-transactions-per-block', 'n-transactions']


# Model train / validation
In order to train and validate the model I'll try several approaches:
- `Default without normalization:` make predictions using the base model
- `Default with normalization:` like the previous one but features are normalized

Then the features that gave on average the most satisfactory results (for each model) are chosen and proceeded with:
- `Hyperparameter tuning:` finding the best parameters to use. 
- `Cross Validation:` validate the performance of the model with the chosen parameters (also here using Block split / Walk forward split)

If the final results are satisfactory, the model will be trained on the whole train / validation set and saved in order to make predictions on the test set.

For each approach the train / validation set will be split according to the chosen splitting method (in order to figure out which one works best for our problem). In this case the `Walk forward time series splits` method will be used: involves using a sliding window approach to create the training and validation sets for each fold. The model is trained on a fixed window of historical data, and then validated on the next observation in the time series. This process is repeated for each subsequent observation, with the window sliding forward one step at a time. 

<img src="https://github.com/CorsiDanilo/bitcoin-price-prediction-with-pyspark/blob/main/notebooks/images/walk-forward-splits.png?raw=1">

In [13]:
# Get splitting parameters based on the choosen splitting method
splitting_info = train_validation_utilities.get_splitting_params(SPLITTING_METHOD)
splitting_info

{'split_type': 'walk_forward_splits',
 'min_obser': 20000,
 'sliding_window': 5000}

## Default
The train / validation set will be splitted based on the splitting method chosen so that the model performance can be seen without any tuning by using different features (normalized and non)

In [14]:
# Get default parameters
params = train_validation_utilities.get_defaults_model_params(MODEL_NAME)
params

{'maxIter': [20], 'maxDepth': [5], 'stepSize': [0.1], 'seed': [42]}

### Without normalization

In [15]:
# Define model and features type
MODEL_TYPE = "default"
FEATURES_NORMALIZATION = False

In [16]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [17]:
# Make predictions by using base features
default_train_results_base_features, default_valid_results_base_features, default_train_pred_base_features, default_valid_pred_base_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:05<00:00,  5.91s/it]


In [18]:
default_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_features,1,"(20000, 5000)","[20, 5, 0.1, 42]",38.025608,1445.946891,28.187303,0.003617,0.999063,0.999063,7.851143
1,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_features,2,"(20000, 5000)","[20, 5, 0.1, 42]",41.580552,1728.942264,29.993176,0.003680,0.998954,0.998954,5.642516
2,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_features,3,"(20000, 5000)","[20, 5, 0.1, 42]",50.991889,2600.172722,38.027498,0.004421,0.999103,0.999103,5.338460
3,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_features,4,"(20000, 5000)","[20, 5, 0.1, 42]",59.080724,3490.531977,40.001244,0.003891,0.998872,0.998872,5.398416
4,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_features,5,"(20000, 5000)","[20, 5, 0.1, 42]",154.340786,23821.078368,84.117335,0.005476,0.999125,0.999125,5.574275
5,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_features,6,"(20000, 5000)","[20, 5, 0.1, 42]",299.777239,89866.392727,194.098198,0.008886,0.999431,0.999431,5.119053
6,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_features,7,"(20000, 5000)","[20, 5, 0.1, 42]",334.162830,111664.797032,240.903323,0.009152,0.999638,0.999638,5.142460
7,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_features,8,"(20000, 5000)","[20, 5, 0.1, 42]",309.491807,95785.178443,231.094857,0.006329,0.999523,0.999523,5.056733
8,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_features,9,"(20000, 5000)","[20, 5, 0.1, 42]",250.451316,62725.861613,189.369944,0.004366,0.999375,0.999375,5.085828
9,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_features,10,"(20000, 5000)","[20, 5, 0.1, 42]",244.020333,59545.923104,189.007307,0.004283,0.999293,0.999293,4.949719


In [19]:
default_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,1,"(20000, 5000)","[20, 5, 0.1, 42]",314.366864,9.882653e+04,135.121327,0.012654,0.747853,0.747651,7.851143
1,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,2,"(20000, 5000)","[20, 5, 0.1, 42]",476.790953,2.273296e+05,397.198468,0.035126,0.360561,0.360049,5.642516
2,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,3,"(20000, 5000)","[20, 5, 0.1, 42]",1686.544411,2.844432e+06,1068.660435,0.073695,0.125174,0.124473,5.338460
3,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,4,"(20000, 5000)","[20, 5, 0.1, 42]",7211.854850,5.201085e+07,5610.942378,0.233261,-1.595551,-1.597630,5.398416
4,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,5,"(20000, 5000)","[20, 5, 0.1, 42]",13389.352236,1.792748e+08,10996.909526,0.247336,-2.280513,-2.283140,5.574275
5,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,6,"(20000, 5000)","[20, 5, 0.1, 42]",2934.082137,8.608838e+06,2219.921972,0.038546,0.544562,0.544197,5.119053
6,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,7,"(20000, 5000)","[20, 5, 0.1, 42]",845.258912,7.144626e+05,532.668255,0.012067,0.990831,0.990824,5.142460
7,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,8,"(20000, 5000)","[20, 5, 0.1, 42]",756.502006,5.722953e+05,534.680763,0.015478,0.932924,0.932870,5.056733
8,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,9,"(20000, 5000)","[20, 5, 0.1, 42]",688.210692,4.736340e+05,495.689252,0.010913,0.944419,0.944375,5.085828
9,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,10,"(20000, 5000)","[20, 5, 0.1, 42]",2252.006206,5.071532e+06,1447.219444,0.023305,0.914250,0.914181,4.949719


In [20]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [21]:
# Make predictions by using base and most additional correlated features
default_train_results_base_and_most_corr_features, default_valid_results_base_and_most_corr_features, default_train_pred_base_and_most_corr_features, default_valid_pred_base_and_most_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:05<00:00,  5.89s/it]


In [22]:
default_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_most_corr_features,1,"(20000, 5000)","[20, 5, 0.1, 42]",36.519948,1333.706600,27.529392,0.003516,0.999136,0.999136,5.057536
1,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_most_corr_features,2,"(20000, 5000)","[20, 5, 0.1, 42]",39.406406,1552.864835,28.224494,0.003477,0.999061,0.999060,4.934102
2,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_most_corr_features,3,"(20000, 5000)","[20, 5, 0.1, 42]",47.906843,2295.065617,35.714626,0.004141,0.999208,0.999208,4.856431
3,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_most_corr_features,4,"(20000, 5000)","[20, 5, 0.1, 42]",50.594343,2559.787590,34.873178,0.003288,0.999173,0.999173,4.825089
4,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_most_corr_features,5,"(20000, 5000)","[20, 5, 0.1, 42]",153.244041,23483.736158,86.768058,0.005753,0.999138,0.999137,4.910501
5,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_most_corr_features,6,"(20000, 5000)","[20, 5, 0.1, 42]",275.356001,75820.927211,182.375984,0.008428,0.999520,0.999520,5.011597
6,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_most_corr_features,7,"(20000, 5000)","[20, 5, 0.1, 42]",323.172865,104440.700427,237.203908,0.008916,0.999662,0.999662,4.927401
7,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_most_corr_features,8,"(20000, 5000)","[20, 5, 0.1, 42]",312.518642,97667.901891,230.266963,0.006294,0.999514,0.999514,4.803581
8,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_most_corr_features,9,"(20000, 5000)","[20, 5, 0.1, 42]",242.708909,58907.614582,184.717213,0.004256,0.999413,0.999413,4.845470
9,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_most_corr_features,10,"(20000, 5000)","[20, 5, 0.1, 42]",242.502819,58807.617228,187.865060,0.004241,0.999302,0.999302,5.205943


In [23]:
default_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,1,"(20000, 5000)","[20, 5, 0.1, 42]",367.204128,1.348389e+05,155.363992,0.014494,0.655970,0.655695,5.057536
1,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,2,"(20000, 5000)","[20, 5, 0.1, 42]",513.654786,2.638412e+05,390.345457,0.035466,0.257860,0.257266,4.934102
2,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,3,"(20000, 5000)","[20, 5, 0.1, 42]",1668.066800,2.782447e+06,1053.132447,0.072626,0.144238,0.143552,4.856431
3,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,4,"(20000, 5000)","[20, 5, 0.1, 42]",7319.028277,5.356817e+07,5754.267759,0.240242,-1.673268,-1.675409,4.825089
4,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,5,"(20000, 5000)","[20, 5, 0.1, 42]",13272.933725,1.761708e+08,10903.172634,0.245209,-2.223714,-2.226295,4.910501
5,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,6,"(20000, 5000)","[20, 5, 0.1, 42]",2945.967742,8.678726e+06,2196.944778,0.038151,0.540864,0.540497,5.011597
6,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,7,"(20000, 5000)","[20, 5, 0.1, 42]",863.466592,7.455746e+05,536.666763,0.012132,0.990432,0.990424,4.927401
7,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,8,"(20000, 5000)","[20, 5, 0.1, 42]",1020.306703,1.041026e+06,709.165770,0.020522,0.877986,0.877888,4.803581
8,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,9,"(20000, 5000)","[20, 5, 0.1, 42]",703.360639,4.947162e+05,512.721108,0.011270,0.941945,0.941899,4.845470
9,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,10,"(20000, 5000)","[20, 5, 0.1, 42]",2199.043691,4.835793e+06,1424.943772,0.023016,0.918236,0.918170,5.205943


In [24]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [25]:
# Make predictions by using base and least additional correlated features
default_train_results_base_and_least_corr_features, default_valid_results_base_and_least_corr_features, default_train_pred_base_and_least_corr_features, default_valid_pred_base_and_least_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:06<00:00,  6.26s/it]


In [26]:
default_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_least_corr_features,1,"(20000, 5000)","[20, 5, 0.1, 42]",36.996044,1368.707252,28.051473,0.003593,0.999113,0.999113,5.303610
1,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_least_corr_features,2,"(20000, 5000)","[20, 5, 0.1, 42]",38.570421,1487.677360,27.981840,0.003446,0.999100,0.999100,5.201030
2,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_least_corr_features,3,"(20000, 5000)","[20, 5, 0.1, 42]",48.991281,2400.145658,36.873018,0.004283,0.999172,0.999172,4.942152
3,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_least_corr_features,4,"(20000, 5000)","[20, 5, 0.1, 42]",43.852536,1923.044895,32.336474,0.003071,0.999379,0.999379,5.039921
4,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_least_corr_features,5,"(20000, 5000)","[20, 5, 0.1, 42]",96.234739,9261.124980,67.312268,0.004865,0.999660,0.999660,5.129959
5,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_least_corr_features,6,"(20000, 5000)","[20, 5, 0.1, 42]",264.773165,70104.829127,175.322007,0.008243,0.999556,0.999556,5.332523
6,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_least_corr_features,7,"(20000, 5000)","[20, 5, 0.1, 42]",328.097279,107647.824274,236.427014,0.008535,0.999651,0.999651,5.725289
7,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_least_corr_features,8,"(20000, 5000)","[20, 5, 0.1, 42]",332.957867,110860.941058,240.049841,0.006575,0.999448,0.999448,4.950864
8,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_least_corr_features,9,"(20000, 5000)","[20, 5, 0.1, 42]",238.383792,56826.832158,180.604133,0.004183,0.999434,0.999433,5.425502
9,GradientBoostingTreeRegressor,default,train,walk_forward_splits,base_and_least_corr_features,10,"(20000, 5000)","[20, 5, 0.1, 42]",234.737051,55101.483088,181.346468,0.004148,0.999346,0.999346,4.950276


In [27]:
default_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,1,"(20000, 5000)","[20, 5, 0.1, 42]",374.213772,1.400359e+05,159.312093,0.014871,0.642710,0.642424,5.303610
1,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,2,"(20000, 5000)","[20, 5, 0.1, 42]",586.260558,3.437014e+05,447.864927,0.040113,0.033227,0.032453,5.201030
2,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,3,"(20000, 5000)","[20, 5, 0.1, 42]",1756.300227,3.084590e+06,1107.216555,0.076328,0.051311,0.050552,4.942152
3,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,4,"(20000, 5000)","[20, 5, 0.1, 42]",8916.437301,7.950285e+07,7711.772074,0.335446,-2.967513,-2.970690,5.039921
4,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,5,"(20000, 5000)","[20, 5, 0.1, 42]",11807.098050,1.394076e+08,9167.209599,0.202006,-1.550991,-1.553034,5.129959
5,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,6,"(20000, 5000)","[20, 5, 0.1, 42]",3644.936918,1.328557e+07,2834.869688,0.048912,0.297146,0.296583,5.332523
6,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,7,"(20000, 5000)","[20, 5, 0.1, 42]",874.931402,7.655050e+05,552.942264,0.012532,0.990176,0.990168,5.725289
7,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,8,"(20000, 5000)","[20, 5, 0.1, 42]",1294.177689,1.674896e+06,957.988711,0.028020,0.803692,0.803535,4.950864
8,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,9,"(20000, 5000)","[20, 5, 0.1, 42]",958.236070,9.182164e+05,679.810447,0.014822,0.892248,0.892162,5.425502
9,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,10,"(20000, 5000)","[20, 5, 0.1, 42]",1922.095326,3.694450e+06,1231.473864,0.019939,0.937534,0.937484,4.950276


### With normalization

In [28]:
# Define model and features type
MODEL_TYPE = "default_norm"
FEATURES_NORMALIZATION = True

In [29]:
# Choose base features
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

In [30]:
# Make predictions by using base features
default_norm_train_results_base_features, default_norm_valid_results_base_features, default_norm_train_pred_base_features, default_norm_valid_pred_base_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:05<00:00,  5.79s/it]


In [31]:
default_norm_train_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_features_norm,1,"(20000, 5000)","[20, 5, 0.1, 42]",68.375629,4675.226610,48.210206,0.006072,0.996972,0.996971,4.919299
1,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_features_norm,2,"(20000, 5000)","[20, 5, 0.1, 42]",63.475602,4029.152039,44.456128,0.005433,0.997562,0.997562,4.753691
2,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_features_norm,3,"(20000, 5000)","[20, 5, 0.1, 42]",68.136728,4642.613712,49.333100,0.005694,0.998398,0.998398,4.783979
3,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_features_norm,4,"(20000, 5000)","[20, 5, 0.1, 42]",73.950445,5468.668323,50.482868,0.004747,0.998233,0.998233,4.819778
4,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_features_norm,5,"(20000, 5000)","[20, 5, 0.1, 42]",138.754070,19252.692011,86.105502,0.006130,0.999293,0.999293,4.857255
5,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_features_norm,6,"(20000, 5000)","[20, 5, 0.1, 42]",353.912710,125254.206159,229.315336,0.010805,0.999206,0.999206,4.868464
6,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_features_norm,7,"(20000, 5000)","[20, 5, 0.1, 42]",467.249220,218321.833253,325.721681,0.011616,0.999293,0.999293,4.766315
7,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_features_norm,8,"(20000, 5000)","[20, 5, 0.1, 42]",504.886825,254910.706065,364.066438,0.009879,0.998732,0.998731,4.744597
8,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_features_norm,9,"(20000, 5000)","[20, 5, 0.1, 42]",471.286632,222111.089574,345.792506,0.008073,0.997786,0.997786,4.819287
9,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_features_norm,10,"(20000, 5000)","[20, 5, 0.1, 42]",447.369196,200139.197322,330.245925,0.007324,0.997623,0.997623,5.002122


In [32]:
default_norm_valid_results_base_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,1,"(20000, 5000)","[20, 5, 0.1, 42]",417.736840,1.745041e+05,215.259326,0.020693,0.554768,0.554411,4.919299
1,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,2,"(20000, 5000)","[20, 5, 0.1, 42]",449.263293,2.018375e+05,406.885386,0.035874,0.432266,0.431811,4.753691
2,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,3,"(20000, 5000)","[20, 5, 0.1, 42]",1813.069694,3.287222e+06,1201.594281,0.083946,-0.011010,-0.011819,4.783979
3,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,4,"(20000, 5000)","[20, 5, 0.1, 42]",7336.842044,5.382925e+07,5765.079982,0.240675,-1.686297,-1.688448,4.819778
4,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,5,"(20000, 5000)","[20, 5, 0.1, 42]",13326.906904,1.776064e+08,10715.617538,0.239061,-2.249985,-2.252587,4.857255
5,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,6,"(20000, 5000)","[20, 5, 0.1, 42]",2938.354367,8.633926e+06,2349.525825,0.041402,0.543234,0.542869,4.868464
6,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,7,"(20000, 5000)","[20, 5, 0.1, 42]",1702.708118,2.899215e+06,1084.058286,0.024285,0.962794,0.962764,4.766315
7,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,8,"(20000, 5000)","[20, 5, 0.1, 42]",2371.813692,5.625500e+06,1456.086322,0.042709,0.340658,0.340130,4.744597
8,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,9,"(20000, 5000)","[20, 5, 0.1, 42]",1126.081417,1.268059e+06,804.092255,0.017565,0.851194,0.851075,4.819287
9,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,10,"(20000, 5000)","[20, 5, 0.1, 42]",2287.763204,5.233860e+06,1657.621595,0.027614,0.911505,0.911434,5.002122


In [33]:
# Choose base and most additional correlated features
CHOSEN_FEATURES = BASE_AND_MOST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_MOST_CORR_FEATURES_LABEL

In [34]:
# Make predictions by using base and most additional correlated features
default_norm_train_results_base_and_most_corr_features, default_norm_valid_results_base_and_most_corr_features, default_norm_train_pred_base_and_most_corr_features, default_norm_valid_pred_base_and_most_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:05<00:00,  5.96s/it]


In [35]:
default_norm_train_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,1,"(20000, 5000)","[20, 5, 0.1, 42]",66.177565,4379.470140,48.829220,0.006152,0.997163,0.997163,5.638201
1,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,2,"(20000, 5000)","[20, 5, 0.1, 42]",60.734840,3688.720808,43.909824,0.005327,0.997768,0.997768,4.948298
2,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,3,"(20000, 5000)","[20, 5, 0.1, 42]",62.863873,3951.866482,45.567684,0.005272,0.998636,0.998636,5.059287
3,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,4,"(20000, 5000)","[20, 5, 0.1, 42]",55.767319,3109.993891,40.202213,0.003801,0.998995,0.998995,5.018034
4,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,5,"(20000, 5000)","[20, 5, 0.1, 42]",105.244178,11076.336935,68.431677,0.004848,0.999593,0.999593,5.007324
5,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,6,"(20000, 5000)","[20, 5, 0.1, 42]",273.450519,74775.186145,177.875129,0.008470,0.999526,0.999526,4.857929
6,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,7,"(20000, 5000)","[20, 5, 0.1, 42]",386.235604,149177.941892,269.728960,0.009933,0.999517,0.999517,4.925286
7,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,8,"(20000, 5000)","[20, 5, 0.1, 42]",528.019661,278804.762079,368.930247,0.009777,0.998613,0.998612,5.211839
8,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,9,"(20000, 5000)","[20, 5, 0.1, 42]",488.932839,239055.320905,361.736288,0.008494,0.997617,0.997617,5.246239
9,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_most_corr_features_norm,10,"(20000, 5000)","[20, 5, 0.1, 42]",429.811007,184737.501722,320.268487,0.007128,0.997806,0.997806,4.992207


In [36]:
default_norm_valid_results_base_and_most_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,1,"(20000, 5000)","[20, 5, 0.1, 42]",449.699438,2.022296e+05,218.167361,0.020815,0.484029,0.483616,5.638201
1,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,2,"(20000, 5000)","[20, 5, 0.1, 42]",499.181353,2.491820e+05,426.683109,0.037665,0.299094,0.298533,4.948298
2,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,3,"(20000, 5000)","[20, 5, 0.1, 42]",1828.047168,3.341756e+06,1215.660456,0.085018,-0.027782,-0.028605,5.059287
3,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,4,"(20000, 5000)","[20, 5, 0.1, 42]",7526.044084,5.664134e+07,5894.253910,0.245860,-1.826631,-1.828895,5.018034
4,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,5,"(20000, 5000)","[20, 5, 0.1, 42]",14791.346292,2.187839e+08,12819.213426,0.296706,-3.003483,-3.006689,5.007324
5,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,6,"(20000, 5000)","[20, 5, 0.1, 42]",3392.586461,1.150964e+07,2784.802531,0.048876,0.391099,0.390611,4.857929
6,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,7,"(20000, 5000)","[20, 5, 0.1, 42]",3062.940843,9.381607e+06,2151.165796,0.052434,0.879603,0.879507,4.925286
7,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,8,"(20000, 5000)","[20, 5, 0.1, 42]",1270.878399,1.615132e+06,1039.506416,0.029524,0.810697,0.810545,5.211839
8,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,9,"(20000, 5000)","[20, 5, 0.1, 42]",1249.779774,1.561949e+06,972.206707,0.021235,0.816707,0.816560,5.246239
9,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,10,"(20000, 5000)","[20, 5, 0.1, 42]",2254.588554,5.083170e+06,1645.236350,0.027488,0.914053,0.913984,4.992207


In [37]:
# Choose base and least additional correlated features
CHOSEN_FEATURES = BASE_AND_LEAST_CORR_FEATURES
CHOSEN_FEATURES_LABEL = BASE_AND_LEAST_CORR_FEATURES_LABEL

In [38]:
# Make predictions by using base and least additional correlated features
default_norm_train_results_base_and_least_corr_features, default_norm_valid_results_base_and_least_corr_features, default_norm_train_pred_base_and_least_corr_features, default_norm_valid_pred_base_and_least_corr_features = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:06<00:00,  6.04s/it]


In [39]:
default_norm_train_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,1,"(20000, 5000)","[20, 5, 0.1, 42]",73.884619,5458.936900,51.774845,0.006551,0.996464,0.996463,4.883177
1,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,2,"(20000, 5000)","[20, 5, 0.1, 42]",74.957549,5618.634133,55.081454,0.006706,0.996601,0.996600,5.017187
2,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,3,"(20000, 5000)","[20, 5, 0.1, 42]",79.743493,6359.024714,57.485825,0.006593,0.997806,0.997805,4.866368
3,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,4,"(20000, 5000)","[20, 5, 0.1, 42]",83.899154,7039.068031,58.516975,0.005594,0.997726,0.997725,4.955006
4,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,5,"(20000, 5000)","[20, 5, 0.1, 42]",110.112002,12124.653047,75.063741,0.005675,0.999555,0.999555,4.806131
5,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,6,"(20000, 5000)","[20, 5, 0.1, 42]",287.910350,82892.369401,194.190096,0.009583,0.999475,0.999475,4.882333
6,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,7,"(20000, 5000)","[20, 5, 0.1, 42]",337.671127,114021.790229,245.003279,0.009313,0.999631,0.999631,4.928586
7,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,8,"(20000, 5000)","[20, 5, 0.1, 42]",450.227997,202705.249250,331.364276,0.008967,0.998991,0.998991,4.979843
8,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,9,"(20000, 5000)","[20, 5, 0.1, 42]",544.853353,296865.176171,397.999533,0.009179,0.997041,0.997041,4.963062
9,GradientBoostingTreeRegressor,default_norm,train,walk_forward_splits,base_and_least_corr_features_norm,10,"(20000, 5000)","[20, 5, 0.1, 42]",561.415472,315187.332339,393.646492,0.008714,0.996257,0.996256,4.905973


In [40]:
default_norm_valid_results_base_and_least_corr_features

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,1,"(20000, 5000)","[20, 5, 0.1, 42]",498.770100,2.487716e+05,379.139096,0.038487,0.365281,0.364772,4.883177
1,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,2,"(20000, 5000)","[20, 5, 0.1, 42]",684.700906,4.688153e+05,564.690273,0.051090,-0.318696,-0.319752,5.017187
2,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,3,"(20000, 5000)","[20, 5, 0.1, 42]",2691.635064,7.244899e+06,1744.729786,0.121476,-1.228223,-1.230007,4.866368
3,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,4,"(20000, 5000)","[20, 5, 0.1, 42]",7377.589335,5.442882e+07,5777.485928,0.241062,-1.716218,-1.718393,4.955006
4,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,5,"(20000, 5000)","[20, 5, 0.1, 42]",13546.695270,1.835130e+08,11243.447083,0.253797,-2.358067,-2.360756,4.806131
5,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,6,"(20000, 5000)","[20, 5, 0.1, 42]",3441.184034,1.184175e+07,2556.185495,0.044617,0.373529,0.373027,4.882333
6,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,7,"(20000, 5000)","[20, 5, 0.1, 42]",5300.175221,2.809186e+07,4540.443581,0.099026,0.639490,0.639201,4.928586
7,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,8,"(20000, 5000)","[20, 5, 0.1, 42]",12339.658113,1.522672e+08,10258.537032,0.293658,-16.846624,-16.860915,4.979843
8,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,9,"(20000, 5000)","[20, 5, 0.1, 42]",9889.231956,9.779691e+07,8979.126255,0.192671,-10.476383,-10.485574,4.963062
9,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,10,"(20000, 5000)","[20, 5, 0.1, 42]",10428.218289,1.087477e+08,9240.284454,0.153781,-0.838719,-0.840192,4.905973


In [41]:
# Define model information and evaluators to show
model_info = ['Model', 'Type', 'Dataset', 'Splitting', 'Features', 'Parameters']
evaluator_lst = ['RMSE', 'MSE', 'MAE', 'MAPE', 'R2', 'Adjusted_R2', 'Time']

In [42]:
# Define the results to show
default_comparison_lst = [default_valid_results_base_features, default_valid_results_base_and_most_corr_features, default_valid_results_base_and_least_corr_features, default_norm_valid_results_base_features, default_norm_valid_results_base_and_most_corr_features, default_norm_valid_results_base_and_least_corr_features]

# Show the comparison table
default_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_comparison_lst])
default_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,"[20, 5, 0.1, 42]",2364.160981,1.604835e+07,1891.399556,0.064908,-0.305974,-0.307020,5.178586
0,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,"[20, 5, 0.1, 42]",2415.117612,1.595036e+07,1930.283391,0.066672,-0.532936,-0.534164,4.961946
0,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,"[20, 5, 0.1, 42]",2736.673530,2.083106e+07,2226.881607,0.081338,-1.825028,-1.827290,5.160749
0,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,"[20, 5, 0.1, 42]",2585.241523,1.654886e+07,2040.650714,0.070092,-0.493966,-0.495162,4.873139
0,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,"[20, 5, 0.1, 42]",2883.238029,1.952624e+07,2358.525547,0.079048,-0.767724,-0.769139,5.023366
0,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,"[20, 5, 0.1, 42]",4845.856536,3.893802e+07,4119.059003,0.124085,-3.070407,-3.073666,4.902149


In [45]:
# Save the best default model results and predicitons
best_default_results = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in [default_valid_results_base_features]])
best_default_predictions = default_valid_pred_base_features
best_default_results

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,"[20, 5, 0.1, 42]",2364.160981,1.604835e+07,1891.399556,0.064908,-0.305974,-0.30702,5.178586


In [46]:
# Define the choosen features
FEATURES_NORMALIZATION = False
CHOSEN_FEATURES = BASE_FEATURES
CHOSEN_FEATURES_LABEL = BASE_FEATURES_LABEL

## Tuned
Once the features and execution method are selected, the model will undergo hyperparameter tuning and cross validation to find the best configuration. 

In [47]:
# Get model grid parameters
params = train_validation_utilities.get_model_grid_params(MODEL_NAME)
params

{'maxIter': [3, 5, 10, 20, 30],
 'maxDepth': [3, 5, 10],
 'stepSize': [0.1, 0.3, 0.5, 0.7],
 'seed': [42]}

### Hyperparameter tuning

In [48]:
# Select the type of feature to be used
MODEL_TYPE = "hyp_tuning"

In [49]:
# Perform hyperparameter tuning
hyp_res = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)
hyp_res

Split: [1/22]: 100%|██████████| 60/60 [05:11<00:00,  5.18s/it]


Best parameters chosen for split [1/22]: [20, 5, 0.7, 42]


Split: [2/22]: 100%|██████████| 60/60 [05:21<00:00,  5.36s/it]


Best parameters chosen for split [2/22]: [10, 3, 0.3, 42]


Split: [3/22]: 100%|██████████| 60/60 [05:42<00:00,  5.70s/it]


...

Split: [21/22]: 100%|██████████| 60/60 [05:33<00:00,  5.56s/it]


Best parameters chosen for split [21/22]: [20, 3, 0.7, 42]


Split: [22/22]: 100%|██████████| 60/60 [05:33<00:00,  5.56s/it]

Best parameters chosen for split [22/22]: [20, 3, 0.7, 42]


,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_features,1,"(20000, 5000)","[20, 5, 0.7, 42]",283.928685,8.061550e+04,140.963023,0.013518,0.794317,0.794152,4.508464
1,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_features,2,"(20000, 5000)","[10, 3, 0.3, 42]",456.056671,2.079877e+05,388.278167,0.034461,0.414967,0.414498,1.700133
2,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_features,3,"(20000, 5000)","[5, 3, 0.5, 42]",1617.745303,2.617100e+06,1019.675167,0.070292,0.195091,0.194447,0.807302
3,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_features,4,"(20000, 5000)","[3, 5, 0.7, 42]",6969.203477,4.856980e+07,5426.548827,0.225738,-1.423829,-1.425770,0.867240
4,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_features,5,"(20000, 5000)","[5, 3, 0.1, 42]",12491.660824,1.560416e+08,10051.314582,0.223749,-1.855374,-1.857660,0.977716
5,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_features,6,"(20000, 5000)","[20, 3, 0.5, 42]",2709.694299,7.342443e+06,2007.504439,0.034886,0.611558,0.611247,3.096148
6,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_features,7,"(20000, 5000)","[20, 5, 0.1, 42]",845.258912,7.144626e+05,532.668255,0.012067,0.990831,0.990824,5.278567
7,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_features,8,"(20000, 5000)","[20, 3, 0.5, 42]",545.883272,2.979885e+05,382.967450,0.010886,0.965074,0.965046,3.180651
8,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_features,9,"(20000, 5000)","[30, 5, 0.3, 42]",677.459317,4.589511e+05,495.407026,0.010886,0.946142,0.946099,7.643772
9,GradientBoostingTreeRegressor,hyp_tuning,valid,walk_forward_splits,base_features,10,"(20000, 5000)","[20, 3, 0.3, 42]",1777.614830,3.159914e+06,1149.855995,0.019278,0.946572,0.946529,3.138526


---
Since during this stage will be used the Block split / Walk forward split method of the dataset I compute a score for each parameter chosen by each split, assigning weights based on:
   * Their `frequency` for each split (if the same parameters are chosen from several splits, these will have greater weight) 
   * The `split` they belong to (the closer the split is to today's date the more weight they will have)
   * Their `RMSE value` for each split (the lower this is, the more weight they will have)
   
   Then, the overall score will be calculated by putting together these three weights for each parameter and the one with the best score will be the chosen parameter.

In [50]:
# Show parameters score
grouped_scores, best_params = train_validation_utilities.choose_best_params(hyp_res)
grouped_scores

,Split weight,RMSE weight,Frequency weight,Final score
Parameters,,,,
"(20, 3, 0.7, 42)",0.977273,0.940823,0.666667,0.612961
"(20, 3, 0.5, 42)",0.439394,0.726430,1.000000,0.319189
"(5, 3, 0.5, 42)",0.477273,0.921439,0.666667,0.293185
"(5, 5, 0.5, 42)",0.772727,0.954434,0.333333,0.245839
"(10, 5, 0.1, 42)",0.727273,0.977325,0.333333,0.236927
"(3, 3, 0.5, 42)",0.909091,0.752152,0.333333,0.227925
"(30, 5, 0.1, 42)",0.590909,0.978234,0.333333,0.192682
"(5, 3, 0.1, 42)",0.545455,0.484222,0.666667,0.176081
"(10, 5, 0.3, 42)",0.545455,0.961837,0.333333,0.174879


In [51]:
# Print best parameters
print(f"Best parameters: {best_params}")

Best parameters: (20, 3, 0.7, 42)


### Cross validation

In [52]:
MODEL_TYPE = "cross_val"

In [53]:
# Get tuned parameters
params = train_validation_utilities.get_best_model_params(best_params, MODEL_NAME)
params

{'maxIter': [20], 'maxDepth': [3], 'stepSize': [0.7], 'seed': [42]}

In [54]:
# Perform cross validation
cv_train_result, cv_valid_result, cv_train_pred, cv_valid_pred = train_validation_utilities.multiple_splits(df, params, splitting_info, MODEL_NAME, MODEL_TYPE, FEATURES_NORMALIZATION, CHOSEN_FEATURES, CHOSEN_FEATURES_LABEL, FEATURES_LABEL, TARGET_LABEL, SLOW_OPERATIONS)

Split: [22/22]: 100%|██████████| 1/1 [00:04<00:00,  4.08s/it]


In [55]:
cv_train_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_features,1,"(20000, 5000)","[20, 3, 0.7, 42]",53.134411,2823.265648,38.669542,0.005038,0.998171,0.998171,3.260236
1,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_features,2,"(20000, 5000)","[20, 3, 0.7, 42]",49.576196,2457.799233,36.406278,0.004479,0.998513,0.998513,3.319584
2,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_features,3,"(20000, 5000)","[20, 3, 0.7, 42]",59.889591,3586.763137,43.880844,0.005179,0.998762,0.998762,3.285819
3,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_features,4,"(20000, 5000)","[20, 3, 0.7, 42]",80.145525,6423.305162,53.195750,0.005054,0.997925,0.997924,3.233754
4,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_features,5,"(20000, 5000)","[20, 3, 0.7, 42]",209.302393,43807.491573,128.442911,0.008695,0.998391,0.998391,3.277201
5,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_features,6,"(20000, 5000)","[20, 3, 0.7, 42]",418.065182,174778.496237,259.585893,0.011709,0.998893,0.998892,3.276337
6,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_features,7,"(20000, 5000)","[20, 3, 0.7, 42]",443.733196,196899.149335,324.184331,0.011670,0.999362,0.999362,3.228767
7,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_features,8,"(20000, 5000)","[20, 3, 0.7, 42]",385.839115,148871.822534,277.021268,0.007574,0.999259,0.999259,3.221987
8,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_features,9,"(20000, 5000)","[20, 3, 0.7, 42]",324.012622,104984.179135,239.501502,0.005485,0.998954,0.998953,3.228413
9,GradientBoostingTreeRegressor,cross_val,train,walk_forward_splits,base_features,10,"(20000, 5000)","[20, 3, 0.7, 42]",307.148576,94340.248010,232.329583,0.005236,0.998880,0.998879,3.222001


In [56]:
cv_valid_result

,Model,Type,Dataset,Splitting,Features,Splits,Train / Validation,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,1,"(20000, 5000)","[20, 3, 0.7, 42]",303.461963,9.208916e+04,138.925530,0.013149,0.765042,0.764854,3.260236
1,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,2,"(20000, 5000)","[20, 3, 0.7, 42]",484.176542,2.344269e+05,425.362823,0.037648,0.340598,0.340070,3.319584
2,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,3,"(20000, 5000)","[20, 3, 0.7, 42]",1687.557712,2.847851e+06,1053.726870,0.072464,0.124122,0.123421,3.285819
3,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,4,"(20000, 5000)","[20, 3, 0.7, 42]",7269.926162,5.285183e+07,5639.645124,0.234211,-1.637519,-1.639632,3.233754
4,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,5,"(20000, 5000)","[20, 3, 0.7, 42]",13443.394590,1.807249e+08,11128.704832,0.251401,-2.307048,-2.309696,3.277201
5,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,6,"(20000, 5000)","[20, 3, 0.7, 42]",3957.484625,1.566168e+07,3169.185313,0.056090,0.171441,0.170777,3.276337
6,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,7,"(20000, 5000)","[20, 3, 0.7, 42]",2437.503191,5.941422e+06,1587.306383,0.039372,0.923752,0.923691,3.228767
7,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,8,"(20000, 5000)","[20, 3, 0.7, 42]",894.574154,8.002629e+05,729.772610,0.021055,0.906204,0.906129,3.221987
8,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,9,"(20000, 5000)","[20, 3, 0.7, 42]",817.329665,6.680278e+05,603.184931,0.013193,0.921608,0.921545,3.228413
9,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,10,"(20000, 5000)","[20, 3, 0.7, 42]",2586.883981,6.691969e+06,1787.511100,0.029369,0.886851,0.886761,3.222001


In [57]:
# Define the results to show
tuned_comparison_lst = [cv_valid_result]

# Show the comparison table
tuned_comparison_lst_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in tuned_comparison_lst])
tuned_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,"[20, 3, 0.7, 42]",2665.683991,1.761518e+07,2144.798023,0.071177,-0.428705,-0.429849,3.223881


# Comparison table
Visualization of model performance at various stages of train / validation

In [58]:
# Concatenate final results into Pandas dataset
final_comparison_lst_df = pd.DataFrame(pd.concat([default_comparison_lst_df, tuned_comparison_lst_df], ignore_index=True))
final_comparison_lst_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,"[20, 5, 0.1, 42]",2364.160981,1.604835e+07,1891.399556,0.064908,-0.305974,-0.307020,5.178586
1,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_most_corr_features,"[20, 5, 0.1, 42]",2415.117612,1.595036e+07,1930.283391,0.066672,-0.532936,-0.534164,4.961946
2,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_and_least_corr_features,"[20, 5, 0.1, 42]",2736.673530,2.083106e+07,2226.881607,0.081338,-1.825028,-1.827290,5.160749
3,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_features_norm,"[20, 5, 0.1, 42]",2585.241523,1.654886e+07,2040.650714,0.070092,-0.493966,-0.495162,4.873139
4,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_most_corr_features_norm,"[20, 5, 0.1, 42]",2883.238029,1.952624e+07,2358.525547,0.079048,-0.767724,-0.769139,5.023366
5,GradientBoostingTreeRegressor,default_norm,valid,walk_forward_splits,base_and_least_corr_features_norm,"[20, 5, 0.1, 42]",4845.856536,3.893802e+07,4119.059003,0.124085,-3.070407,-3.073666,4.902149
6,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,"[20, 3, 0.7, 42]",2665.683991,1.761518e+07,2144.798023,0.071177,-0.428705,-0.429849,3.223881


# Model accuracy

Since predicting the price accurately is very difficult I also saw how good the models are at predicting whether the price will go up or down in this way:

For each prediction let's consider the actual market-price, next-market-price and our predicted next-market-price (prediction).
I compute whether the current prediction is correct (1) or not (0):

$$ 
prediction\_is\_correct
= 
\begin{cases}
0 \text{ if [(market-price > next-market-price) and (market-price < prediction)] or [(market-price < next-market-price) and (market-price > prediction)]} \\
1 \text{ if [(market-price > next-market-price) and (market-price > prediction)] or [(market-price < next-market-price) and (market-price < prediction)]}
\end{cases}
$$

After that I count the number of correct prediction:
$$ 
correct\_predictions
= 
\sum_{i=0}^{total\_rows} prediction\_is\_correct
$$

Finally I compute the percentage of accuracy of the model:
$$
\\ 
accuracy 
= 
(correct\_predictions / total\_rows) 
* 100
$$

In [59]:
# Convert the pandas dataset to a PySpark dataset
best_default_pred_spark = spark.createDataFrame(best_default_predictions)
validated_pred_spark = spark.createDataFrame(cv_valid_pred)

# Compute model accuracy
default_accuracy = train_validation_utilities.model_accuracy(best_default_pred_spark)
validated_accuracy = train_validation_utilities.model_accuracy(validated_pred_spark)

# Shows whether features are normalised or not
if FEATURES_NORMALIZATION:
    NEW_CHOSEN_FEATURES_LABEL = CHOSEN_FEATURES_LABEL + "_norm"
    CHOSEN_FEATURES_LABEL = NEW_CHOSEN_FEATURES_LABEL
    
# Saving accuracy data into dataframe
accuracy_data = {
    'Model': MODEL_NAME,
    'Features': CHOSEN_FEATURES_LABEL,
    'Splitting': SPLITTING_METHOD,
    'Accuracy (default)': default_accuracy,
    'Accuracy (tuned)': validated_accuracy
}
accuracy_data_df = pd.DataFrame(accuracy_data, index=['Model'])

print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (default): {default_accuracy:.2f}%")
print(f"Percentage of correct predictions for {MODEL_NAME} with {CHOSEN_FEATURES_LABEL} and {SPLITTING_METHOD} (tuned): {validated_accuracy:.2f}%")

Percentage of correct predictions for GradientBoostingTreeRegressor with base_features and walk_forward_splits (default): 49.03%
Percentage of correct predictions for GradientBoostingTreeRegressor with base_features and walk_forward_splits (tuned): 50.01%


In [60]:
# Concatenate default and tuned results
default_tuned_results = [best_default_results, cv_valid_result]
default_tuned_results_df = pd.concat([train_validation_utilities.model_comparison(results, model_info, evaluator_lst) for results in default_tuned_results])
default_tuned_results_df

,Model,Type,Dataset,Splitting,Features,Parameters,RMSE,MSE,MAE,MAPE,R2,Adjusted_R2,Time
0,GradientBoostingTreeRegressor,default,valid,walk_forward_splits,base_features,"[20, 5, 0.1, 42]",2364.160981,1.604835e+07,1891.399556,0.064908,-0.305974,-0.307020,5.178586
0,GradientBoostingTreeRegressor,cross_val,valid,walk_forward_splits,base_features,"[20, 3, 0.7, 42]",2665.683991,1.761518e+07,2144.798023,0.071177,-0.428705,-0.429849,3.223881


# Saving final results

In [61]:
# Save all final comparison results
final_comparison_lst_df.to_csv(ALL_MODEL_RESULTS, index=False)

In [62]:
# Save relevant results (default and tuned results)
default_tuned_results_df.to_csv(REL_MODEL_RESULTS, index=False)

In [63]:
# Saving accuracy results
accuracy_data_df.to_csv(MODEL_ACCURACY_RESULTS, index=False)

In [64]:
# Export notebook in html format (remember to save the notebook and change the model name)
if LOCAL_RUNNING:
    !jupyter nbconvert --to html 4-walk-forward-split_{MODEL_NAME}.ipynb --output 4-walk-forward-split_{MODEL_NAME} --output-dir='./exports'

d:\Documents\Repository\BDC\project\.bdc\Lib\site-packages\traitlets\traitlets.py:2548: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use './exports' instead of "'./exports'" if you require traitlets >=5.
  warn(
[NbConvertApp] Converting notebook 4-walk-forward-split_GradientBoostingTreeRegressor.ipynb to html
[NbConvertApp] WARNING | Alternative text is missing on 1 image(s).
[NbConvertApp] Writing 546727 bytes to exports\4-walk-forward-split_GradientBoostingTreeRegressor.html
